In [1]:
#Importación de librerías
import os
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
#Cargar el modelo preentrenado
base_model = load_model('cifar10.h5')

#Congelar capas para conservar características
for layer in base_model.layers[:-5]:
    layer.trainable = False

c:\Users\jeman\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jeman\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
#Ajustar las últimas capas del modelo
n_classes = 4

x = base_model.layers[-2].output                                                #Conectar a la penúltima capa del modelo base
x = Dense(256, activation='relu', name='custom_dense_stl10')(x)                 #Nombre único para la primera capa densa
output = Dense(n_classes, activation='softmax', name='custom_output_stl10')(x)  #Nombre único para la capa de salida
model = Model(inputs=base_model.input, outputs=output)

In [4]:
#Cargar datos desde el directorio
data_dir = '4Clases/'
all_images = []
all_labels = []

#Recorrer las clases dentro del directorio principal
for label in range(n_classes):
    class_dir = os.path.join(data_dir, f'class_{label}')
    #Obtener lista de imágenes en cada clase
    image_names = os.listdir(class_dir)
    #Completar las rutas de las imágenes
    image_paths = [os.path.join(class_dir, img_name) for img_name in image_names if img_name.endswith('.jpg')]
    #Agregar a la lista de todas las imágenes y etiquetas
    all_images.extend(image_paths)
    all_labels.extend([label] * len(image_paths))

In [ ]:
#Crear conjuntos de datos
batch_size = 32
img_height = 32
img_width = 32

train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.3, random_state=123)

val_images, test_images, val_labels, test_labels = train_test_split(
    test_images, test_labels, test_size=0.5, random_state=123)

#Procesamiento de imágenes
def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, size=(img_height, img_width))
    img = tf.cast(img, tf.float32) / 255.0  # Normalizar a [0,1]
    return img, label

#Crear datasets de entrenamiento, validación y prueba
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

#Calcular la media y la desviación estándar de los datos de entrenamiento
mean = 0
std = 0
num_batches = 0

for images, _ in train_dataset:
    batch_mean = tf.reduce_mean(images, axis=[0, 1, 2])
    batch_std = tf.math.reduce_std(images, axis=[0, 1, 2])
    mean += batch_mean
    std += batch_std
    num_batches += 1

mean /= num_batches
std /= num_batches

#Normalizar los datos usando z-score y codificar las etiquetas
def normalize_and_encode_img(image, label):
    image = (image - mean) / (std + 1e-7)
    label = tf.one_hot(label, n_classes)
    return image, label

train_dataset = train_dataset.map(normalize_and_encode_img)
val_dataset = val_dataset.map(normalize_and_encode_img)
test_dataset = test_dataset.map(normalize_and_encode_img)

In [ ]:
#Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Definir callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

#Entrenar el modelo
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
#Evaluar el modelo
loss, accuracy = model.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Precisión: {accuracy}")

# Predicciones y etiquetas verdaderas para el conjunto de prueba
predictions = model.predict(test_dataset)
y_pred = np.argmax(predictions, axis=1)
y_true = np.concatenate([y for _, y in test_dataset], axis=0)
y_true = np.argmax(y_true, axis=1)  # Convertir y_true a su forma original

# Calcular precisión global del nuevo modelo usando sklearn
precision_global = accuracy_score(y_true, y_pred)
print("Precisión global del nuevo modelo:", precision_global)

#Guardar el modelo entrenado
model.save('modelo.h5')